In [1]:
import pandas as pd

## Data

In [2]:
data = pd.read_excel('DC_Channel.xlsx')
data

,attribution_technical,attribution_survey,account_creation_date
0,facebook,facebook,2020-01-24 21:44:16
1,facebook,facebook,2020-03-01 15:44:35
2,organic,facebook,2019-12-07 16:37:06
3,search,tv,2020-01-27 16:09:32
4,discovery,youtube,2019-10-05 12:57:07
...,...,...,...
227623,facebook,facebook_organic,2019-11-17 14:12:33
227624,google_organic,referral,2019-12-06 18:02:13
227625,facebook,facebook,2019-12-21 19:40:44
227626,facebook,facebook,2020-01-17 23:58:51


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227628 entries, 0 to 227627
Data columns (total 3 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   attribution_technical  227628 non-null  object        
 1   attribution_survey     224984 non-null  object        
 2   account_creation_date  227628 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 5.2+ MB


In [4]:
channel_spend = pd.read_excel('5-Channel-Spend.xlsx',index_col=0)
channel_spend

,bing,display,facebook,search,youtube
Date,,,,,
201907,400,12,9000,13000,90
201908,900,20,13000,18500,180
201909,1000,13,10500,19000,100
201910,1100,19,17000,24000,130
201911,1300,29,23000,25000,550
201912,300,31,16000,38000,900
202001,2100,148,11000,41000,4360
202002,3700,94,14000,44000,2420


## Reserve only date and month

In [5]:
data['account_creation_date'] = data['account_creation_date'].dt.date
data['account_creation_date'] = (pd.DatetimeIndex(data['account_creation_date']).year)*100 + (pd.DatetimeIndex(data['account_creation_date']).month)
data['account_creation_date']

0         202001
1         202003
2         201912
3         202001
4         201910
           ...  
227623    201911
227624    201912
227625    201912
227626    202001
227627    201911
Name: account_creation_date, Length: 227628, dtype: int64

## Channel spend - attribution_survey

In [6]:
df_survey = pd.DataFrame(data, columns=['attribution_survey','account_creation_date'])
df_survey

,attribution_survey,account_creation_date
0,facebook,202001
1,facebook,202003
2,facebook,201912
3,tv,202001
4,youtube,201910
...,...,...
227623,facebook_organic,201911
227624,referral,201912
227625,facebook,201912
227626,facebook,202001


In [7]:
channel_list = ['bing', 'display', 'facebook', 'search', 'youtube']
time_list = [201907,201908,201909,201910,201911,201912,202001,202002]

In [8]:
# Create empty dataframe
survey_result = pd.DataFrame(columns=['bing', 'display', 'facebook', 'search', 'youtube'],index=time_list)

In [9]:
for c in channel_list:
    count_list = []
    for t in time_list:
        count = len(df_survey[(df_survey['attribution_survey']==c) & (df_survey['account_creation_date']==t)])
        count_list.append(count)
    survey_result[c] = count_list

survey_result

,bing,display,facebook,search,youtube
201907,0,0,14451,536,85
201908,0,0,15399,786,264
201909,0,0,11702,783,209
201910,0,0,13418,853,274
201911,0,0,16930,864,312
201912,0,0,13394,1032,245
202001,0,0,10157,1151,725
202002,0,0,11809,1344,546


In [10]:
spend_list = []
for c in channel_list:
    spend_list.append(channel_spend[c].sum())

spend_list

[10800, 366, 113500, 222500, 8730]

In [11]:
convert_list = []
for c in survey_result:
    convert_list.append(survey_result[c].sum())

convert_list

[0, 0, 107260, 7349, 2660]

### Average CAC

In [12]:
avg_cac_list = []
for i in range(0,5):
    if convert_list[i] == 0:
        avg_cac_list.append(0)
    else:
        avg_cac = round(spend_list[i]/convert_list[i],2)
        avg_cac_list.append(avg_cac)

avg_cac_list

[0, 0, 1.06, 30.28, 3.28]

### Marginal CAC

In [13]:
# Create empty dataframe
marg_cac = pd.DataFrame(columns=['bing', 'display', 'facebook', 'search', 'youtube'],index=time_list)

In [14]:
for c in channel_list:
    marg_cac_list = []
    for t in time_list:
        if survey_result[c][t] == 0:
            marg_cac_list.append(0)
        else:
            marg_cac_value = round(channel_spend[c][t]/survey_result[c][t],2)
            marg_cac_list.append(marg_cac_value)
    marg_cac[c] = marg_cac_list

marg_cac

,bing,display,facebook,search,youtube
201907,0,0,0.62,24.25,1.06
201908,0,0,0.84,23.54,0.68
201909,0,0,0.90,24.27,0.48
201910,0,0,1.27,28.14,0.47
201911,0,0,1.36,28.94,1.76
201912,0,0,1.19,36.82,3.67
202001,0,0,1.08,35.62,6.01
202002,0,0,1.19,32.74,4.43


In [15]:
marg_cac.to_csv('ca_survey.csv')